In [37]:
import pandas as pd


df_men = pd.read_csv('C:\\Users\\mmladenov\\Desktop\\github_repos\\cov_mortality\\source_data\\cz_life_expectancy_men.csv')
df_women = pd.read_csv('C:\\Users\\mmladenov\\Desktop\\github_repos\\cov_mortality\\source_data\\cz_life_expectancy_women.csv')

In [38]:
df_women.dropna(how='any',axis=0,inplace=True)
df_women['age'] = df_women['age'].astype('int')

df_men.dropna(how='any',axis=0,inplace=True)
df_men['age'] = df_men['age'].astype('int')

both_sexes = df_men.merge(df_women, on='age',how='left')
both_sexes = both_sexes.round(2)

In [40]:
both_sexes

,age,men,women
0,0,76.33,82.10
1,1,75.55,81.28
2,2,74.57,80.30
3,3,73.58,79.31
4,4,72.58,78.32
...,...,...,...
101,101,1.74,1.73
102,102,1.66,1.64
103,103,1.59,1.56
104,104,1.53,1.50
